In [ ]:
Model = "https://huggingface.co/KoboldAI/LLaMA2-13B-Tiefighter-GGUF/resolve/main/LLaMA2-13B-Tiefighter.Q4_K_M.gguf"
Layers = 43
ContextSize = 4096
ForceRebuild = False

import os
if not os.path.isfile("/opt/bin/nvidia-smi"):
  raise RuntimeError("⚠️Colab did not give you a GPU due to usage limits, this can take a few hours before they let you back in. Check out https://lite.koboldai.net for a free alternative (that does not provide an API link but can load KoboldAI saves and chat cards) or subscribe to Colab Pro for immediate access.⚠️")

%cd /content
!git clone https://github.com/LostRuins/koboldcpp
%cd /content/koboldcpp
kvers = !(cat koboldcpp.py | grep 'KcppVersion = ' | cut -d '"' -f2)
kvers = kvers[0]
if ForceRebuild:
  kvers = "force_rebuild"
!echo Finding prebuilt binary for {kvers}
!wget -O dlfile.tmp https://kcppcolab.concedo.workers.dev/?{kvers} && mv dlfile.tmp koboldcpp_cublas.so
!test -f koboldcpp_cublas.so && echo Prebuilt Binary Exists || echo Prebuilt Binary Does Not Exist
!test -f koboldcpp_cublas.so && echo Build Skipped || make koboldcpp_cublas LLAMA_CUBLAS=1 LLAMA_PORTABLE=1
!cp koboldcpp_cublas.so koboldcpp_cublas.dat
!apt install aria2 -y
!aria2c -x 10 -o model.ggml --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none $Model
!python koboldcpp.py model.ggml --usecublas 0 mmq --multiuser --gpulayers $Layers --contextsize $ContextSize --quiet --remotetunnel
